# EDA

In [25]:
import numpy as np
import pandas as pd

import os
import glob

In [34]:
BASE_DIR = os.path.join(os.getcwd(), os.pardir)
DATA_DIR = os.path.join(BASE_DIR, 'data')

TYPE_LABEL = {'clicks':0, 'carts':1, 'orders':2}


## ファイルimport

In [37]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(TYPE_LABEL).astype('int8')
    return df

In [40]:
# # ================
# # sample_submission
# # ================
# path_sample_submission = os.path.join(BASE_DIR, 'data', 'original_data', 'sample_submission.csv')
# sample_sub = pd.read_csv(path_sample_submission)

# # ================
# # train/test parquet
# # ================
# train_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'train_parquet', '*'))
# test_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'test_parquet', '*'))
# # data_cache_train = {}
# # data_cache_test = {}
# # for p in train_paths: data_cache_train[p] = read_file_to_cache(p)
# # for p in test_paths: data_cache_test[p] = read_file_to_cache(p)

# ================
# validation file
# ================


